# Install dependencies

See [install_dependencies.ipynb](install_dependencies.ipynb).

# How to use this Notebook
1. Click *Kernel* -> *Restart Kernel and Run All Cells*.

# Import dependencies

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from PIL import Image, ImageDraw, ImageFont
import io
import cv2
from matplotlib import pyplot as plt
import numpy as np
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, opencv_image_to_mpl, VideoDataGenerator, generate_image_bytes

In [3]:
imp.reload(video);

# Define Pravega stream parameters

In [4]:
gateway = 'hop-claudio-ubuntu18-desktop.solarch.lab.emc.com:54672'
scope = 'examples'
stream = 'generated-video'

# Initialize connection to Pravega GRPC Gateway

In [5]:
pravega_channel = grpc.insecure_channel(gateway)
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

# Create data generator

In [10]:
imp.reload(video);
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, IndexedStream, opencv_image_to_mpl, VideoDataGenerator

In [11]:
video_data_generator = VideoDataGenerator(pravega_client, scope, stream, fps=3.0, verbose=False)

In [8]:
video_data_generator.truncate_stream()

# Generate video frames

In [12]:
video_data_generator.write_generated_video(num_frames=10000)

KeyboardInterrupt: 

# View sample generated image

In [ ]:
img_bytes = video_data_generator.generate_image_bytes(frame_number=12345)
print('JPEG image size: %d bytes' % len(img_bytes))
image_bytes_array = np.frombuffer(img_bytes, dtype=np.uint8)
image_array = cv2.imdecode(image_bytes_array, cv2.IMREAD_UNCHANGED)
plt.imshow(opencv_image_to_mpl(image_array));